In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

In [ ]:
velib_path = "../data/interim/data_20230902.pkl"

In [ ]:
velib_df = pd.read_pickle(velib_path)
# velib_df = velib_df.set_index("datetime").sort_index()
velib_df = velib_df.sort_values(by=["datetime", "station_id"])

In [ ]:
velib_df.columns

In [ ]:
velib_gdf = gpd.GeoDataFrame(
    velib_df, geometry=gpd.points_from_xy(x=velib_df["lon"], y=velib_df["lat"])
)

In [ ]:
douzieme_shp = gpd.read_file("../data/external/douzieme_polygon.shx")

In [ ]:
sim_geo = gpd.GeoSeries(douzieme_shp.loc[0, "geometry"]).simplify(
    tolerance=0.001
)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j)

In [ ]:
# velib_gdf.iloc[:100].plot(column="num_bikes_available", legend=True)

In [ ]:
station_geo_info = velib_gdf[~velib_gdf["station_id"].duplicated(keep="last")]
station_geo_info = station_geo_info[
    station_geo_info["geometry"].within(douzieme_shp.loc[0, "geometry"])
]
marker_list = []
for idx, row in station_geo_info.iterrows():
    if row.geometry.is_empty:
        continue
    marker = folium.Marker(
        location=(row.geometry.y, row.geometry.x),
        popup=f"""Nom de la station: {row["name"]}
        <br>Nombre de vélos disponibles: {row["num_bikes_available"]}
        <br>Nombre de places disponibles: {row["num_docks_available"]}
        <br>Id de la station: {row["station_id"]}""",
        tooltip=row["name"],
    )
    marker_list.append(marker)

In [ ]:
paris_center = [48.862, 2.346]
paris = folium.Map(location=paris_center, zoom_start=12)
marker_cluster = MarkerCluster().add_to(paris)
for marker in marker_list:
    marker.add_to(marker_cluster)

geo_j.add_to(paris)
paris

In [ ]:
def plot_single_station(station_id):
    station_df = velib_df[velib_df["station_id"] == station_id]
    fig = px.line(station_df, x="datetime", y="num_bikes_available")
    fig.update_layout(title=f"Station {station_id}")
    fig.show()

In [ ]:
plot_single_station(124183944)

In [ ]:
velib_df["use_rate"] = velib_df["num_bikes_available"] / velib_df["capacity"]

In [ ]:
station_name_list = velib_df["name"].drop_duplicates()

In [ ]:
gare_station_name_list = station_name_list[
    station_name_list.str.contains("Gare").fillna(False)
]

In [ ]:
print(velib_df.shape)
velib_df.head()

In [ ]:
# px.line(velib_df[velib_df["name"].isin(gare_station_name_list.iloc[:10])], x="datetime", y="use_rate", color="name")

In [ ]:
velib_df